In [2]:
import os
import pandas as pd
import glob

# Helpers

In [3]:
def get_lines_with_not_zero_count(target_df, cols=['I1', 'I2', 'I6', 'I7', 'I8', 'I9']):
    """
    Returns a new dataframe with only the rows containing at least 1 violation
    """
    return target_df[(target_df[cols].sum(axis=1) > 0)]

def get_stats(manual_analysis_df, target_df, name):
    cols=['I1', 'I2', 'I6', 'I7', 'I8', 'I9']
    target_violations = get_lines_with_not_zero_count(target_df, cols=cols)
    
    res = {}
    res['name'] = name
    res['contracts_number'] = target_df.shape[0]

    counter_fp = 0
    counter_tp = 0
    number_violations = 0
    
    for c in cols:
        violation_col = get_lines_with_not_zero_count(target_df, [c])
        merged_df = manual_analysis_df.merge(violation_col, on=['address'], how='inner', indicator=True)
        
        res[c] = violation_col.shape[0]
        c_index = c.lower()
        res[c + "_TP"] = len(merged_df[(merged_df[c_index] == "TP")])
        res[c + "_FP"] = len(merged_df[(merged_df[c_index] == "FP")])
        res[c + "_X"] = len(merged_df[(merged_df[c_index] != "FP") & (merged_df[c_index] != "TP")])
        
        counter_fp += res[c + "_FP"]
        counter_tp += res[c + "_TP"]
        number_violations += res[c]
        
        # DEBUG
        try:
            assert(res[c + "_FP"] + res[c + "_TP"] + res[c + "_X"] == res[c])
        except:
            merged_df = manual_analysis_df.merge(violation_col, on=['address'], how='outer', indicator=True)
            print(name, merged_df[merged_df['_merge'] == 'right_only'])       

        
    res['total_TP'] = counter_tp
    res['total_FP'] = counter_fp
    res['violation_number'] = number_violations


    return res

# Import Manual Analysis Report

In [4]:
socrates_findings = '../experiments_manual_analysis/manual-analysis.csv'

In [5]:
socrates_df = pd.read_csv(socrates_findings)
socrates_df.columns = [x.lower() for x in socrates_df.columns]
socrates_df = socrates_df.drop(columns=[x for x in socrates_df.columns if 'unnamed' in x])
socrates_df = socrates_df[socrates_df['experiment type'] != "no-one"]
socrates_df.head()

,address,token name,experiment type,invariants violated,i1,i2,i6,i7,i8,i9,explanation,report,note
0,0xf3e70642c28f3f707408c56624c2f30ea9f9fce3,AlbosToken,testfull10,I2,NaN,FP,NaN,NaN,NaN,68,violation is caused by their business logic,FP,NaN
1,0xf7d3320c4676d11d67338b766a9df99996d19777,MKC,testfull10,I7,NaN,NaN,NaN,TP,NaN,NaN,allowance -= safeSub(x - y),TP,NaN
2,0xb57919aebb30812ae188dbe238bc907d56ba4a3a,EveryCoin,testfull10,"I2, I7",NaN,TP,NaN,FP,NaN,NaN,"constructor name mismatch (I2), burnFrom inter...",TP,NaN
3,0x45e5997a4a69ca3d8eb38892bd7d5dad8eadea2b,TCZToken,testfull10,I6,NaN,NaN,TP,NaN,NaN,NaN,"transferFrom with their own implementation, no...",TP,NaN
4,0x4594a218d3149743758b08574f1f532cb790e268,VTEXP,testfull10,I2,NaN,TP,NaN,NaN,NaN,NaN,"wrong transfer logic, money should not be subt...",TP,NaN


# Import CSVs

In [6]:
testfull10 = "../experiments_raw_csvs/testfull10.csv"
testfull1 = "../experiments_raw_csvs/testfull1.csv"
testrandom10 = "../experiments_raw_csvs/testrandom10.csv"
testoverflow10 = "../experiments_raw_csvs/testoverflow10.csv"
testboundary10 = "../experiments_raw_csvs/testboundary10.csv"
testcombinedrandom10 = "../experiments_raw_csvs/testcombinedrandom10.csv"

In [7]:
testfull10_df = pd.read_csv(testfull10)
testfull10_df['address'].str.lower()
testfull10_violated_df = get_lines_with_not_zero_count(testfull10_df, cols=['I1', 'I2', 'I6', 'I7', 'I8', 'I9'])
testfull10_violated_df.to_csv('../experiments_only_violated_csvs/testfull10_violated.csv')

testfull1_df = pd.read_csv(testfull1)
testfull1_df['address'].str.lower()
testfull1_violated_df = get_lines_with_not_zero_count(testfull1_df, cols=['I1', 'I2', 'I6', 'I7', 'I8', 'I9'])
testfull1_violated_df.to_csv('../experiments_only_violated_csvs/testfull1_violated.csv')

testrandom10_df = pd.read_csv(testrandom10)
testrandom10_df['address'].str.lower()
testrandom10_violated_df = get_lines_with_not_zero_count(testrandom10_df, cols=['I1', 'I2', 'I6', 'I7', 'I8', 'I9'])
testrandom10_violated_df.to_csv('../experiments_only_violated_csvs/testrandom10_violated.csv')

testoverflow10_df = pd.read_csv(testoverflow10)
testoverflow10_df['address'].str.lower()
testoverflow10_violated_df = get_lines_with_not_zero_count(testoverflow10_df, cols=['I1', 'I2', 'I6', 'I7', 'I8', 'I9'])
testoverflow10_violated_df.to_csv('../experiments_only_violated_csvs/testoverflow10_violated.csv')

testboundary10_df = pd.read_csv(testboundary10)
testboundary10_df['address'].str.lower()
testboundary10_violated_df = get_lines_with_not_zero_count(testboundary10_df, cols=['I1', 'I2', 'I6', 'I7', 'I8', 'I9'])
testboundary10_violated_df.to_csv('../experiments_only_violated_csvs/testboundary10_violated.csv')

testcombinedrandom10_df = pd.read_csv(testcombinedrandom10)
testcombinedrandom10_df['address'].str.lower()
testcombinedrandom10_violated_df = get_lines_with_not_zero_count(testcombinedrandom10_df, cols=['I1', 'I2', 'I6', 'I7', 'I8', 'I9'])
testcombinedrandom10_violated_df.to_csv('../experiments_only_violated_csvs/testcombinedrandom10_violated.csv')

# Stats


In [8]:
testfull10_stat = get_stats(socrates_df, testfull10_df, 'testfull10')
testfull1_stat = get_stats(socrates_df, testfull1_df, 'testfull1')
testrandom10_stat = get_stats(socrates_df, testrandom10_df, 'testrandom10')
testoverflow10_stat = get_stats(socrates_df, testoverflow10_df, 'testoverflow10')
testboundary10_stat = get_stats(socrates_df, testboundary10_df, 'testboundary10')
testcombinedrandom10_stat = get_stats(socrates_df, testcombinedrandom10_df, 'testcombinedrandom10')

stats = [testfull10_stat, testfull1_stat, testrandom10_stat, testoverflow10_stat, testboundary10_stat, testcombinedrandom10_stat]
stats_df = pd.DataFrame(stats)
stats_df.to_csv('../experiments_stats_csvs/all_experiments_stats.csv')
stats_df.to_latex()

'\\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrrrrrlrrr}\n\\toprule\n{} &  I1 &  I1\\_FP &  I1\\_TP &  I1\\_X &   I2 &  I2\\_FP &  I2\\_TP &  I2\\_X &  I6 &  I6\\_FP &  I6\\_TP &  I6\\_X &  I7 &  I7\\_FP &  I7\\_TP &  I7\\_X &  I8 &  I8\\_FP &  I8\\_TP &  I8\\_X &  I9 &  I9\\_FP &  I9\\_TP &  I9\\_X &  contracts\\_number &                  name &  total\\_FP &  total\\_TP &  violation\\_number \\\\\n\\midrule\n0 &  10 &      0 &     10 &     0 &  134 &      2 &     61 &    71 &   3 &      2 &      1 &     0 &   4 &      3 &      1 &     0 &   3 &      1 &      2 &     0 &  21 &      1 &     20 &     0 &              1059 &            testfull10 &         9 &        95 &               175 \\\\\n1 &  10 &      0 &     10 &     0 &  131 &      1 &     59 &    71 &   0 &      0 &      0 &     0 &   1 &      1 &      0 &     0 &   2 &      1 &      1 &     0 &  20 &      0 &     20 &     0 &              1059 &             testfull1 &         3 &        90 &               164 \\\\\n2 &   0 &      

# Uniquely Identified violations

In [9]:
# List of all identified violations [OK]
# Address I1, I2 ... I3
# 0x....D,0,  1, ..., 3

# TODO found_matrix
# 0x1, 1, 0, full10
# 0x1, 1, 0, random10
# 0x1, 0, 0  overflow10
# 0x1, 0, 1, boundary10

# then do something like:
# if group(address).sum('I1', axis=1).count() == 1 # someone, uniquely identified this
#      group(address).get(address, where I1 == 1)

def get_found_matrix(df):
    found_matrix = df.copy(deep=True)
    
    found_matrix['I1'] = (found_matrix['I1'] > 0).astype(int)
    found_matrix['I2'] = (found_matrix['I2'] > 0).astype(int)
    found_matrix['I6'] = (found_matrix['I6'] > 0).astype(int)
    found_matrix['I7'] = (found_matrix['I7'] > 0).astype(int)
    found_matrix['I8'] = (found_matrix['I8'] > 0).astype(int)
    found_matrix['I9'] = (found_matrix['I9'] > 0).astype(int)
        
    return found_matrix

fm_testfull10_df = get_found_matrix(testfull10_violated_df)
fm_testfull10_df['configuration'] = 'testfull10'

fm_testfull1_df = get_found_matrix(testfull1_violated_df)
fm_testfull1_df['configuration'] = 'testfull1'

fm_testrandom10_df = get_found_matrix(testrandom10_violated_df)
fm_testrandom10_df['configuration'] = 'testrandom10'

fm_testoverflow10_df = get_found_matrix(testoverflow10_violated_df)
fm_testoverflow10_df['configuration'] = 'testoverflow10'

fm_testboundary10_df = get_found_matrix(testboundary10_violated_df)
fm_testboundary10_df['configuration'] = 'testboundary10'

fm_testcombinedrandom10_df = get_found_matrix(testcombinedrandom10_violated_df)
fm_testcombinedrandom10_df['configuration'] = 'testcombinedrandom10'

In [10]:
def filter_uniquely_identified(df):
    new_table = {}

    for index, row in without_duplicates.iterrows():
        address = row['address']
        configuration = row['configuration']

        if address not in new_table:
            new_table[address] = {
                                 'address': address,
                                 'name': row['name'],
                                 'I1': [], 'I2': [], 'I6': [], 'I7': [], 'I8': [], 'I9': []}

        merged_row = new_table[address]

        for i in ['I1', 'I2', 'I6', 'I7', 'I8', 'I9']:
            if row[i] == 1:
                merged_row[i].append(row['configuration'])   

    items = new_table.values()
    filtered = []
    for item in items:
        for i in ['I1', 'I2', 'I6', 'I7', 'I8', 'I9']:
            if len(item[i]) == 1:
                item[i] = item[i][0]
            else:
                item[i] = len(item[i])
        filtered.append(item)

    return pd.DataFrame(items)

In [12]:
fm_all_but_1bot = pd.concat([fm_testfull10_df, fm_testrandom10_df,  fm_testoverflow10_df, fm_testboundary10_df, fm_testcombinedrandom10_df ])
without_duplicates = fm_all_but_1bot.drop_duplicates(subset=['address', 'I1', 'I2', 'I6', 'I7', 'I8', 'I9'], keep=False)
uniquely_identified_all10 = filter_uniquely_identified(without_duplicates)
uniquely_identified_all10 = uniquely_identified_all10[['address', 'name', 'I1', 'I2', 'I6', 'I7', 'I8', 'I9']] #RE-ORDER
uniquely_identified_all10.to_csv('../experiments_stats_csvs/uniquely_identified_all10.csv')
uniquely_identified_all10

,address,name,I1,I2,I6,I7,I8,I9
0,0x3aca398197478e1f911e4edd8ff7b4d608b98a3a,EncryptedToken,2,testcombinedrandom10,0,testcombinedrandom10,0,2
1,0x0b76544f6c413a555f309bf76260d1e02377c02a,INT,2,testrandom10,0,2,0,4
2,0xb57919aebb30812ae188dbe238bc907d56ba4a3a,EveryCoin,0,testfull10,0,0,0,0
3,0xb87972abbfc219664e4a2f52954f5b9cedf7ea33,ExpressCoin,2,testfull10,0,0,0,0
4,0x9ef0eb9d97e4e7c8612d4e5aaf9201e35bd3670b,EAI_TokenERC,testfull10,testfull10,0,0,0,0
5,0x7f4404db6a3f4d3cfb41d5920d81d44d64759235,Yumerium,2,3,0,testcombinedrandom10,0,2
6,0x5b0ea5b1735046b2e10ee8feb404d2d5f1e1ad4b,EMBCToken,0,0,0,testfull10,0,0
7,0xcfac2916ec118a0252a7766c513ee7c71b384b5e,HoryouToken,0,testfull10,0,0,testfull10,0
8,0xe7d7b37e72510309db27c460378f957b1b04bd5d,EMPR,2,2,0,0,0,0
9,0x219218f117dc9348b358b8471c55a073e5e0da0b,GRX,2,0,0,0,0,testfull10


In [13]:
fm_full_10bot = pd.concat([fm_testfull10_df, fm_testfull1_df])
without_duplicates = fm_full_10bot.drop_duplicates(subset=['address', 'I1', 'I2', 'I6', 'I7', 'I8', 'I9'], keep=False)
uniquely_identified_full10 = filter_uniquely_identified(without_duplicates)
uniquely_identified_full10 = uniquely_identified_full10[['address', 'name', 'I1', 'I2', 'I6', 'I7', 'I8', 'I9']] #RE-ORDER
uniquely_identified_full10.to_csv('../experiments_stats_csvs/uniquely_identified_full10.csv')
uniquely_identified_full10

,address,name,I1,I2,I6,I7,I8,I9
0,0xf3e70642c28f3f707408c56624c2f30ea9f9fce3,AlbosToken,0,testfull10,0,0,0,0
1,0x56f527c3f4a24bb2beba449ffd766331da840ffa,BTYCToken,0,2,0,0,testfull10,0
2,0x0b76544f6c413a555f309bf76260d1e02377c02a,INT,2,testfull1,0,0,0,2
3,0xf7d3320c4676d11d67338b766a9df99996d19777,MKC,0,0,0,testfull10,0,0
4,0xb57919aebb30812ae188dbe238bc907d56ba4a3a,EveryCoin,0,testfull10,0,0,0,0
5,0x45e5997a4a69ca3d8eb38892bd7d5dad8eadea2b,TCZToken,0,0,testfull10,0,0,0
6,0xb87972abbfc219664e4a2f52954f5b9cedf7ea33,ExpressCoin,2,testfull10,0,0,0,0
7,0x9ef0eb9d97e4e7c8612d4e5aaf9201e35bd3670b,EAI_TokenERC,testfull10,testfull10,0,0,0,0
8,0x47dd62d4d075dead71d0e00299fc56a2d747bebb,EqualToken,0,2,0,0,0,testfull10
9,0xeb7c20027172e5d143fb030d50f91cece2d1485d,EBTC,0,0,testfull10,testfull10,0,0


In [14]:
uniquely_identified_all10.to_latex()

'\\begin{tabular}{lllllllll}\n\\toprule\n{} &                                     address &                                 name &          I1 &                    I2 &                    I6 &                    I7 &          I8 &                    I9 \\\\\n\\midrule\n0  &  0x3aca398197478e1f911e4edd8ff7b4d608b98a3a &                       EncryptedToken &           2 &  testcombinedrandom10 &                     0 &  testcombinedrandom10 &           0 &                     2 \\\\\n1  &  0x0b76544f6c413a555f309bf76260d1e02377c02a &                                  INT &           2 &          testrandom10 &                     0 &                     2 &           0 &                     4 \\\\\n2  &  0xb57919aebb30812ae188dbe238bc907d56ba4a3a &                            EveryCoin &           0 &            testfull10 &                     0 &                     0 &           0 &                     0 \\\\\n3  &  0xb87972abbfc219664e4a2f52954f5b9cedf7ea33 &                          

In [15]:
uniquely_identified_full10.to_latex()

'\\begin{tabular}{lllllllll}\n\\toprule\n{} &                                     address &          name &          I1 &          I2 &          I6 &          I7 &          I8 &          I9 \\\\\n\\midrule\n0  &  0xf3e70642c28f3f707408c56624c2f30ea9f9fce3 &    AlbosToken &           0 &  testfull10 &           0 &           0 &           0 &           0 \\\\\n1  &  0x56f527c3f4a24bb2beba449ffd766331da840ffa &     BTYCToken &           0 &           2 &           0 &           0 &  testfull10 &           0 \\\\\n2  &  0x0b76544f6c413a555f309bf76260d1e02377c02a &           INT &           2 &   testfull1 &           0 &           0 &           0 &           2 \\\\\n3  &  0xf7d3320c4676d11d67338b766a9df99996d19777 &           MKC &           0 &           0 &           0 &  testfull10 &           0 &           0 \\\\\n4  &  0xb57919aebb30812ae188dbe238bc907d56ba4a3a &     EveryCoin &           0 &  testfull10 &           0 &           0 &           0 &           0 \\\\\n5  &  0x45e5997a4a